#    APRICOT: Advanced Platform for Reproducible Infrastructures in the Cloud via Open Tools

This notebook provide a complete example of reproducible experimentation in the field of life sciences. The objective is to determine the best reconstruction parameters for a simulated Positron Emission Tomography (PET) scanner system. For that purpose, the source code of two required programs has been distributed with this notebook. The first one, "reconstructor" is an implementation based on the OPLM algorithm for image reconstruction of PET systems. The second, "evaluateImage", gets a reconstructed image as input and compares it with the expected one and returns a set of image quality parameters. The input data is a simulated acquisition of a PET system formed by 3 rings with 20 detector modules each one. The simulated data has been provided via a public S3 bucket (https://s3.amazonaws.com/grycap/datasets/apricot/reconstruction/data.txz).

# Experimentation

Medical scanners, like most physical detectors, measure raw data that must be post-processed to obtain an interpretable result. In particular, for medical scanners based in Positron Emission Tomography (PET) or Computed Tomography (CT), the final result is, usually, a patient image to be interpreted by the physician to develop a diagnostic. 

Focusing on PET and CT cases, there exists a great variety of iterative and analytic image reconstruction algorithms [0][1] [2][3], most of them based on maximum likelihood method [4].  These reconstruction algorithms have a set  of  variable  parameters  such  as  number  and  size  of  voxels  in  the  Field  Of View (FOV), number of iterations, number of partitioned data chunks, weight parameters, filter iterations and weight, etc.  Obviously, the final reconstruction quality  and  speed  will  depend  on  how  accurate  are  the  selected  parameters. Furthermore, the accuracy of selected parameters depend on the scanner system (geometry, energy resolution, scanned object, etc.).  Indeed, the importance of reconstruction parameters on medical image has been studied for different kind of scanners in many publications, such as [5] [6] [7] [8].

Achieving the best parameters for our specific system and algorithm is desirable not only for medical diagnostics but to perform accurate comparison of reconstruction methods and scanner capabilities. That comparison is crucial to select and create new scanner systems using simulated data to study their theoretical performance. However, the number of possible parameters combinations grows as indicated in (1).

\begin{equation}
\prod_{i=1}^{n_{param}} N_i\;,
\end{equation}

where $N_i$ is the number of possible values of parameter number $i$ and $n_{param}$ the number of variable parameters. So, even performing a multiparametric study with few parameters requires a significant computational effort. APRICOT has been used in this experimentation to deploy and manage the required infrastructure to perform a multiparametric study on a modified implementation of "OPL-EM" reconstruction algorithm for PET systems described in the work by Reader et al. [1]. 

The input data used for this experimentation has been obtained simulating a PET system formed by three rings of $20$ detector modules each one. The simulations have been done using a self developed routines to perform PET system simulations with the Monte-Carlo (MC) code PENELOPE [9]. We will measure the reconstruction time and a set of quality metrics to determine the best parameters to achieve the required agreement between time spent for reconstruction and image quality. This metrics are, "Root-Mean-Square Error" (RMSE), "Peak Signal to Noise Ratio" (PSNR), "Normalized Root Mean Square Distance" (NRMSD) and "Normalized Mean Absolute Distance" (NMAD), which equations are listed below,

\begin{equation}
    RMSE = \sqrt{\frac{1}{N} \sum_{m=1}^{N} (v(m) - v_{true}(m))^{2}}
    \label{eq:RMSE}
\end{equation}

\begin{equation}
    PSNR = 10 \cdot log_{10}\left( \frac{\max(v_{true})^2}{\frac{1}{N} \sum_{m=1}^{N} (v(m) -     v_{true}(m))^{2}}\right) 
    \label{eq:PSNR}
\end{equation}

\begin{equation}
    NRMSD = \sqrt{\frac{\sum_{m=1}^{N} (v(m) - v_{true}(m))^{2}}{\sum_{m=1}^{N} (\bar{v}_{true} - v_{true}(m))^{2}}}
    \label{eq:NRMSD}
\end{equation}

\begin{equation}
    NMAD = \frac{\sum_{m=1}^{N} |v(m) - v_{true}(m)|}{\sum_{m=1}^{N} |v_{true} (m)|}
    \label{eq:NMAD}
\end{equation}

At metric equations, $v(m)$ denotes the voxel number $m$ of the considered image and the subindex $true$ indicate that is the real image. In this example we will plot the reconstruction time and the RMSE, but editing the code any metric can be ploted.

## Setting up the system

For this experimentation we need to perform a multiparametric analysis to obtain the best image reconstruction parameters. So, the first step is infrastructure deployment. The chosen infrastructure is a batch cluster with 5 working nodes and ubuntu 18.04 LTS images as OS. Working nodes have 2 GB of RAM and 20 GB of disk space. Also, we recommend to assign, at least, two CPUs to the front-end node for this experiment. To reproduce the experiment deploy this infrastructure using the APRICOT deploy extension in one of the supported Cloud providers. You will need valid access credentials to your Cloud provider. Once the cluster has been configured, you can follow the experimentation.

Store the cluster and user names

In [ ]:
clusterName = "reconstruction"

In [ ]:
username = "ubuntu"

We need to load the python module with the APRICOT magics to manage our clusters. This step can be avoided if the module is loaded by default on each notebook.

In [ ]:
%reload_ext apricot_magic

Once the infrastructure has been deployed and configured, check it using %apricot_ls magic

In [ ]:
%apricot_ls

If the infrastructure deployment fails, we can get the output log with the following instruction,

In [ ]:
%apricot_log $clusterName

## Preparing data and programs

Now, we must provide the necessary data and software for our analisys:

1- Raw simulated data

2- Comparision program code

3- Reconstruction program code


All required source codes can be upload easily from our computer using the ''%apricot_upload'' instruction. However, the input data with raw scanner simulated detections is stored externally because of its size. These data has been stored in a public Amazon S3 bucket and, thus, it can be downloaded using "curl". First, check if the "input" folder is at the current directory:

In [ ]:
%%bash
ls

This folder contains the source codes, upload them to the front-end node of the cluster.

In [ ]:
%apricot_upload $clusterName input /home/$username

Download the input data to the front-end node of the cluster using "curl". This step may take a few minutes

In [ ]:
%apricot exec $clusterName curl https://s3.amazonaws.com/grycap/datasets/apricot/reconstruction/data.txz --output data.txz

Finally, extract the input data.

In [ ]:
%apricot exec $clusterName tar -xvf data.txz

Check if all required files are in the "input" folder

In [ ]:
%apricot exec $clusterName ls input

Now, we need to compile the source codes. All the necessary compilers and cmake tools should be installed at configuration initialization.

Compile the reconstruction and comparision programs

In [ ]:
%apricot exec $clusterName cd input/reconstructor_code && bash install.sh && cp reconstructor ../

In [ ]:
%apricot exec $clusterName cd input && g++ -o evaluateImage evaluateImage.cpp -O2

Check if the executables have been created (evaluateImage and reconstructor)

In [ ]:
%apricot exec $clusterName ls input

Now we have all the necessary files at cluster. This has been configured automatically with NFS, so the '/home' directory is shared by all the working nodes and the front-end node.

We need also a folder to store the results. Create a folder named 'results' for that purpose.

In [ ]:
%apricot exec $clusterName mkdir results

## Executing jobs

Now, it is time to execute the computational experiment. For simplicity, this multiparametric study only uses three variable parameters. However, this can be extended to any number of parameter ranges. First of all, we need to specify each parameter interval and step size. To avoid large execution times, we will use large step sizes for all ranges. With the provided ranges, the total number of executed jobs will be 15.

In [ ]:
minNvox_xy = 20
maxNvox_xy = 250
stepNvox_xy = 50

minNvox_z = 20
maxNvox_z = 250
stepNvox_z = 100

nChunksMin = 5
nChunksMax = 5
chunkStep = 1

Now, use %apricot_genMPid function to obtain an identifier for the specified ranges. We will use this identifier to repeat the experimentation keeping the results of previous runs.

In [ ]:
ID = %apricot_genMPid $minNvox_xy $maxNvox_xy $stepNvox_xy $minNvox_z $maxNvox_z $stepNvox_z $nChunksMin $nChunksMax $chunkStep

In [ ]:
print(ID)

Create a specific folder for this run with the previous ID.

In [ ]:
%apricot exec $clusterName mkdir results/$ID

In [ ]:
%apricot exec $clusterName ls results

Launch the jobs using ''%apricot_runMP'' and the local script ''script.sh''. This step can be delayed several minutes until all workers have been configured.

In [ ]:
%apricot_runMP $clusterName script.sh /home/$username $minNvox_xy $maxNvox_xy $stepNvox_xy $minNvox_z $maxNvox_z $stepNvox_z $nChunksMin $nChunksMax $chunkStep
            

Check if all the jobs have been finished.

In [ ]:
%apricot exec $clusterName squeue

When no jobs appear in the tasks queue, execute post-processing script

In [ ]:
%apricot exec $clusterName bash input/getResults.sh $ID

## Getting results

Once the results of our multiparametric study have been processed, download them to plot the corresponding graphs using "%apricot_download" instruction

In [ ]:
resultsFilename = "results-" + ID + ".dat"
%apricot_download $clusterName $resultsFilename .

In [ ]:
%%bash

ls

Read and parse the results file.

In [ ]:
fileIn = open(resultsFilename,"r")

data = fileIn.read()

# Extract data lines
data = data.strip().split('\n')
# Remove header line
data.pop(0)

# Extract input data
XYnvox = []
Znvox = []
chunks = []
userTimeMin = []
userTimeSec = []
systemTimeMin = []
systemTimeSec = []
RMSE = []
PSNR = []
NRMSD = []
NMAD = []

for line in data:
    line = " ".join(line.split())
    words = line.strip().split(' ')
    XYnvox.append(float(words[0]))
    Znvox.append(float(words[1]))
    chunks.append(float(words[2]))
    userTimeMin.append(float(words[3]))
    userTimeSec.append(float(words[4]))
    systemTimeMin.append(float(words[5]))
    systemTimeSec.append(float(words[6]))
    RMSE.append(float(words[7]))
    PSNR.append(float(words[8]))
    NRMSD.append(float(words[9]))
    NMAD.append(float(words[10]))
    
    
fileIn.close()

Import the numpy and plot libraries. If the required packages are not installed in your system, execute the following statement:

In [ ]:
%%bash
python3 -m pip install --user numpy scipy matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

userTime = np.add(np.multiply(userTimeMin,60.0),userTimeSec)
systTime = np.add(np.multiply(systemTimeMin,60.0),systemTimeSec)
totalTime = userTime + systTime

Plot the results:

In [ ]:

nChunks = 5.0
subXY = []
subZ = []
subTimes = []
subRMSE = []
subPSNR = []
subNRMSD = []
subNMAD = []

subRMSE_zoom = []
subXY_zoom = []
subZ_zoom = []
nVoxCut = 60

for i in list(range(len(XYnvox))):
    if nChunks == chunks[i]:
        subXY.append(XYnvox[i])
        subZ.append(Znvox[i])
        subTimes.append(totalTime[i])
        subRMSE.append(RMSE[i])
        subPSNR.append(PSNR[i])
        subNRMSD.append(NRMSD[i])
        subNMAD.append(NMAD[i])
        if XYnvox[i] > nVoxCut and Znvox[i] > nVoxCut:
            subXY_zoom.append(XYnvox[i])
            subZ_zoom.append(Znvox[i])
            subRMSE_zoom.append(RMSE[i])
        
Axpad = 280        
fig = plt.figure()
#plt.rcParams["figure.figsize"] = [100,100]
#plt.rcParams.update({'font.size': 128})

ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('nº voxels x-y')
ax.set_ylabel('nº voxels z')
ax.set_zlabel('time (s)')

ax.xaxis.labelpad = Axpad
ax.yaxis.labelpad = Axpad
ax.zaxis.labelpad = Axpad


ax.plot_trisurf(subXY,subZ,subTimes)
plt.show()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('nº voxels x-y')
ax.set_ylabel('nº voxels z')
ax.set_zlabel('rmse')

ax.xaxis.labelpad = Axpad
ax.yaxis.labelpad = Axpad
ax.zaxis.labelpad = Axpad

ax.plot_trisurf(subXY,subZ,subRMSE)
plt.show()


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('nº voxels x-y')
ax.set_ylabel('nº voxels z')
ax.set_zlabel('rmse')

ax.xaxis.labelpad = Axpad
ax.yaxis.labelpad = Axpad
ax.zaxis.labelpad = Axpad

ax.plot_trisurf(subXY_zoom,subZ_zoom,subRMSE_zoom)
plt.show()


Previous graphs show the RMSE and reconstruction time for each parameter combination used in our study. In a real case study, these results should provide the best parameters to achieve the required agreement between reconstruction speed and image quality. However this is only a simplified example to show APRICOT functionality. Now, we can repeat the experiment with different parameter ranges or plot different results.

## Delete infrastructure

Once finished the experimentation, do not forget to destroy the cluster.

In [ ]:
%apricot destroy $clusterName

# Conclusions

This document includes infrastructure specifications, data storage, experimentation execution, results gathering and infrastructure termination for a example computational experimentation. Thus it can be reproduced easily distributing the experimentation notebook together with the required source codes.

# References

[0] Xuan Liu, Claude Comtat, Christian Michel, Paul Kinahan, Michel Defrise, and David Townsend. Comparison of 3-d reconstruction with 3d-osem and with fore + osem for pet. IEEE TRANSACTIONS ON MEDICAL IMAGING, 2001

[1]  Andrew  J.  Reader,   Stijn  Ally,   Filippos  Bakatselos,   Roido  Manavaki, Richard J. Walledge, Alan P. Jeavons, Peter J. Julyan, Sha Zhao, David L. Hastings, and Jamal Zweit.  One-pass list-mode em algorithm for high-resolution 3-d pet image reconstruction into large arrays. IEEE  TRANSACTIONS ON NUCLEAR SCIENCE, 2002.

[2]  Sarabjeet Singh, Mannudeep K. Kalra, Jiang Hsieh, Paul E. Licato, Synho Do, Homer H. Pien, Michael A. Blake. Abdominal ct: Comparison of adaptive  statistical  iterative  and  filtered  back  projection  reconstruction  techniques. Radiology, 2010.

[3] Chillarón,  Mónica.,  Vidal,  Vicent,  Verdú,  Gumersindo. Ct  image  reconstruction withsuitesparseqr factorization package. Radiation  Physics  and Chemistry, 2019.

[4] L.A. Sheep and Y.Vardi.  Maximum likelihood reconstruction for emission tomography. IEEE TRANSACTIONS ON MEDICAL IMAGING, 1982.

[5] Jin Mo Goo, Trongtum Tongdee, Ranista Tongdee, Kwangjae Yeo, Charles F. Hildebolt, Kyongtae T. Bae. Volumetric measurement of synthetic lung nodules  with  multi–detector  row  ct:  Effect  of  various  image  reconstruction  parameters  and  segmentation  thresholds  on  measurement  accuracy. Radiology, 2005.

[6] James  G.  Ravenel,  William  M.  Leue,  Paul  J.  Nietert,  James  V.  Miller, Katherine K. Taylor, Gerard A. Silvestri.  Pulmonary nodule volume:  Effects of reconstruction parameters on automated measurements—a phantom study. Radiology, 2008.

[7] Yue-Houng Hu, Bo Zhao, Wei Zhao.  Image artifacts in digital breast tomosynthesis:  Investigation  of  the  effects  of  system  geometry  and  reconstruction  parameters  using  a  linear  system  approach. Medical  Physics, 2008.

[8] Maria Lyra, Agapi Ploussi. Filtering in spect image reconstruction. Journal of Biomedical Imaging, 2011.

[9] Salvat F. Penelope. a code system for monte carlo simulation of electron and photon transport. Issy-Les-Moulineaux: OECD  Nuclear  Energy  Agengy, 2014.